# Minimal CLAS UI Elements
> Creating the minimal gradio app with state, chat, and chat download

In this notebook, we create a minimized version of the current app to be deployed onto Huggingface Space, which has the following components: 

- Student interface: 
    A chat interface where students interact with the chatbot that has had the instructor-designed prompt given to it but they cannot see the prompt. At the end of the conversation, entire convo is made available as downloadable JSON such that students can download it and turn it in to Brightspace.

In [ ]:
#| default_exp BasicInteraction

First, we'll start by loading our own libraries. Keep in mind that if you're on Colab, you need to replace "token" below with your GitHub token.

In [ ]:
# run this code only if you're using Google Colab
#! pip install pip install git+https://ghusername:<token>@github.com/vanderbilt-data-science/lo-achievement.git

In [ ]:
## run this code only if you're local and developing
import os, sys

# get parent of current working directory
parent_dir = os.path.dirname(os.getcwd())

#append to path
sys.path.append(parent_dir)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import os
import gradio as gr
from ai_classroom_suite.UIBaseComponents import *

/workspaces/lo-achievement/ai_classroom_suite/UIBaseComponents.py:354: GradioUnusedKwargWarning: You have unused kwarg parameters in Box, please remove them: {'scale': 1}
  with gr.Box(elem_id="sources-container", scale=1):


## Interface Helpers and Functionality

In the next section, we'll create some helper functions to make sure we're able to create the interface we want, and that it behaves nicely. 

### Student Interface Chatbot Functions ###

In [ ]:
#| export
def get_tutor_reply(chat_tutor): 
  chat_tutor.get_direct_tutor_reply()
  return gr.update(value="", interactive=True), chat_tutor.conversation_memory, chat_tutor

def get_conversation_history(chat_tutor):
    return chat_tutor.conversation_memory, chat_tutor

def initialize_mdl_wrapper(chat_tutor, openai_auth=None, **mdl_chain_kwargs):
  
  if 'OPENAI_API_KEY' not in os.environ.keys() or not os.environ.get("OPENAI_API_KEY"):
      #print('Nope')
      gr.Error('This app will not run successfully without an OPENAI_API_KEY as an environment variable. Please contact your administrator \
                and tell them to set the OPENAI_API_KEY environment variable.')
    
  if 'SECRET_PROMPT' not in os.environ.keys() or not os.environ.get("SECRET_PROMPT"):
      #print('Nada')
      gr.Warning('Your app currently does not have a System Message. Please contact your administrator \
                  and tell them to set the System Message.')
        
  chat_tutor.value = initialize_basic_model(chat_tutor.value, openai_auth=openai_auth, **mdl_chain_kwargs)

  if chat_tutor.value.tutor_chain is None:
     gr.Warning('Your model seems to have failed to initialize. Did you set the OPENAI_API_KEY environment variable?')

  return chat_tutor

# The User Interface

Below, we put all of this information together with the actual formatting of the user interface.

### Note for Instructors ###

You need to provide an OpenAI API Key as well as a Secret Prompt to initialize the model. 

- To set the API Key, in the hosted app on Huggingface Space, go to ``Settings -> Variables and Secrets -> Secrets``, then replace ``OPENAI_API_KEY`` value with your key.
If you haven't created one already, visit [platform.openai.com/account/api-keys](https://platform.openai.com/account/api-keys) to sign up for an account and get your personal API key. 

- To set the Secret Prompt, in the hosted app on Huggingface Space, go to ``Settings -> Variables and Secrets -> Secrets``, then replace ``SECRET_PROMPT`` value with your key.

In [ ]:
# For debugging purposes in the Jupyter notebook, you probably want to set these, otherwise things get annoying.
#os.environ['OPENAI_API_KEY'] = ''
#os.environ['SECRET_PROMPT'] = ''

In [ ]:
#| export
        
### User Interfaces ###
with gr.Blocks() as BasicInteractionDemo:
    #initialize tutor (with state)
    study_tutor = gr.State(SlightlyDelusionalTutor())

    if 'SECRET_PROMPT' in os.environ:
        study_tutor.value.set_system_message(os.environ.get("SECRET_PROMPT"))

    # Student chatbot interface
    gr.Markdown("""
    ## Chat with the Model
    Description here
    """)
    
    """
    API Authentication functionality
    Instead of ask students to provide key, the key is now provided by the instructor. 
    To permanently set the key, go to Settings -> Variables and secrets -> Secrets, 
    then replace OPENAI_API_KEY value with whatever openai key of the instructor.
    """
    api_input = gr.Textbox(show_label=False, type="password", value=os.environ.get("OPENAI_API_KEY"), visible=False)

    # The instructor will provide a secret prompt/persona to the tutor
    instructor_prompt = gr.Textbox(label="Verify your prompt content", value = os.environ.get("SECRET_PROMPT"), visible=False)
    
    # Placeholders components
    text_input_none = gr.Textbox(visible=False)
    file_input_none = gr.File(visible=False)
    instructor_input_none = gr.TextArea(visible=False)
    learning_objectives_none = gr.Textbox(visible=False)

    # Initialize situation with API key so no click is necessary
    study_tutor = initialize_mdl_wrapper(study_tutor, api_input.value)

    with gr.Row(equal_height=True):
        with gr.Column(scale=2):
            chatbot = gr.Chatbot()
            with gr.Row():
                user_chat_input = gr.Textbox(label="User input", scale=9)
                user_chat_submit = gr.Button("Ask/answer model", scale=1)

    # First add user's message to the conversation history
    # Then get reply from the tutor and add that to the conversation history
    user_chat_submit.click(
        fn = add_user_message, inputs = [user_chat_input, study_tutor], outputs = [user_chat_input, chatbot, study_tutor], queue=False
    ).then(
        fn = get_tutor_reply, inputs = [study_tutor], outputs = [user_chat_input, chatbot, study_tutor], queue=True
    )

    user_chat_input.submit(
        fn = add_user_message, inputs = [user_chat_input, study_tutor], outputs = [user_chat_input, chatbot, study_tutor], queue=False
    ).then(
        fn = get_tutor_reply, inputs = [study_tutor], outputs = [user_chat_input, chatbot, study_tutor], queue=True
    )

    # Download conversation history file
    with gr.Box():
        gr.Markdown("""
        ## Export Your Chat History
        Export your chat history as a .json, .txt, or .csv file
        """)
        with gr.Row():
            export_dialogue_button_json = gr.Button("JSON")
            export_dialogue_button_txt = gr.Button("TXT")
            export_dialogue_button_csv = gr.Button("CSV")
    
        file_download = gr.Files(label="Download here", file_types=['.json', '.txt', '.csv'], type="file", visible=False)
    
    export_dialogue_button_json.click(save_json, study_tutor, file_download, show_progress=True)
    export_dialogue_button_txt.click(save_txt, study_tutor, file_download, show_progress=True)
    export_dialogue_button_csv.click(save_csv, study_tutor, file_download, show_progress=True)

In [ ]:
BasicInteractionDemo.queue().launch(server_name='0.0.0.0', server_port=7860)

Running on local URL:  http://0.0.0.0:7860

To create a public link, set `share=True` in `launch()`.


A little helper in case your ports are open and you just want to close them all. If this doesn't work, restart your IDE.

In [ ]:
gr.close_all()

In [ ]:
# TODO: Currently, the instructor prompt is handled as text input and stored in the vector store (and in the learning objective),
# which means the tutor now is still a question-answering tutor who viewed the prompt as context (but not really acting based on it). 
# We need to find a way to provide the prompt directly to the model and set its status. 